In [2]:
import pandas as pd
import numpy as np
from apyori import apriori  

In [3]:
df = pd.read_csv("../data/location1_cleaned.csv")
df_prods = pd.read_csv("../data_engineering/product_image_mapping.csv")

In [4]:
del df['product_sku']
del df['display_description']
del df['location_name']

In [5]:
df.head(10)

,order_id,order_time(UTC),items_id,items_name,item_price,quantity,served_for,item_category_type
0,841222,2018-07-31 19:02:57,2255,Bud Light,7.50,1,Lunch,Alcoholic Beverage
1,841222,2018-07-31 19:02:57,3038,"Shrimp, Fish and Chicken Platter",15.99,1,Lunch,Entree
2,841222,2018-07-31 19:02:57,3038,"Shrimp, Fish and Chicken Platter",15.99,1,Lunch,Entree
3,841222,2018-07-31 19:02:57,2252,Assorted Fountain Beverages - Regular,3.29,1,Lunch,Beverage
4,841222,2018-07-31 19:02:57,3036,Shrimp Platter,12.49,1,Lunch,Entree
5,841222,2018-07-31 19:02:57,3036,Shrimp Platter,12.49,1,Lunch,Entree
6,841231,2018-07-31 19:05:20,3236,Toy Story Land Cupcake,4.99,1,Lunch,Dessert
7,841231,2018-07-31 19:05:20,3236,Toy Story Land Cupcake,4.99,1,Lunch,Dessert
8,841237,2018-07-31 19:08:43,2716,Chicken Breast Strips,9.49,1,Lunch,Entree
9,841237,2018-07-31 19:08:43,2253,Assorted Fountain Beverages - Large,3.69,1,Lunch,Beverage


In [6]:
typical_order_size = df.groupby('order_id').count()['items_name'].mean()
print(f"Average order size: {typical_order_size:.2f}")

Average order size: 4.88


In [12]:
ss = set(list(df['served_for']))
print(ss)

print(len(df[ df['served_for'] == "Lunch"]))
print(len(df[ df['served_for'] == "Dinner"]))

{'Dinner', 'Lunch'}
58449
9107


In [7]:
print(df['quantity'].max())
# so it appears that we always have quantity one and 
# multiple instances of the same item have their own line in the order

1


In [14]:
#Categories:
cats = list(set(list(df_prods['category'])))
cats

print(df_prods[ df_prods['category'] == 'Side'])


      id                      name  \
0   1813              French Fries   
2   1819          Vegetarian Chili   
3   1820  New England Clam Chowder   
10  1838     Green Beans & Carrots   
25  2205              French Fries   
30  2221               Steak Fries   
31  2222  Fire-roasted Corn Medley   
32  2234      Hummus and Flatbread   
64  3166         Macaroni & Cheese   
77  3289  Southwestern Steak Fries   
82  3407               Green Beans   
87  3591               Hushpuppies   
88  3592            Vegetable Rice   

                                   display_desc category  \
0                                           NaN     Side   
2                                           NaN     Side   
3                                           NaN     Side   
10                                          NaN     Side   
25                                          NaN     Side   
30                                          NaN     Side   
31                                          NaN

In [95]:
# most popular items
ydf = (df.groupby('items_id').count()['order_id']).reset_index(drop=False)
ydf.rename(columns={'order_id':'count'},inplace=True)
df_prods.rename(columns={'id':'items_id'},inplace=True)
zdf = pd.merge(ydf,df_prods,on='items_id',how='right')
zdf = zdf.sort_values(['count'],ascending=False)
zdf.head(5)

,items_id,count,name,display_desc,category,location,img
20,2253,7113.0,Assorted Fountain Beverages - Large,Assorted Coca-Cola® Offerings,Beverage,Columbia Harbor House,bev_fountain_beverages.jpeg
19,2252,5484.0,Assorted Fountain Beverages - Regular,Assorted Coca-Cola® Offerings,Beverage,Columbia Harbor House,bev_fountain_beverages.jpeg
9,2237,4800.0,Mediterranean Salad with Chicken,tossed in a Mediterranean Vinaigrette and serv...,Entree,Columbia Harbor House,salad_mediterranean_salad_chicken.png
33,3038,4432.0,"Shrimp, Fish and Chicken Platter",served with Steak Fries and Coleslaw,Entree,Columbia Harbor House,fried_shrimp_fish_chicken.png
29,2716,4382.0,Chicken Breast Strips,"served with your choice of fresh Green Beans, ...",Entree,Columbia Harbor House,fried_chicken-strips.png


In [90]:
pdf = df[ ['order_id','items_id']].copy()
pdf['value'] = 1
pdf.head()

,order_id,items_id,value
0,841222,2255,1
1,841222,3038,1
2,841222,3038,1
3,841222,2252,1
4,841222,3036,1


In [73]:
mdf = pd.pivot_table(pdf,index='order_id',columns=['items_id'],aggfunc=np.max)
mdf.fillna(0.0,inplace=True)

In [74]:
mdf.head()

value                                              ...             \
items_id  2205 2207 2208 2210 2218 2221 2222 2234 2236 2237 ...  4838 4840   
order_id                                                    ...              
841222     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 ...   0.0  0.0   
841231     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 ...   0.0  0.0   
841237     0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0 ...   0.0  0.0   
841328     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 ...   0.0  0.0   
841329     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 ...   0.0  0.0   

                                                  
items_id 4841 4842 5142 5143 5388 5445 5446 5469  
order_id                                          
841222    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
841231    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
841237    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
841328    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
841329    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 144 columns]

In [77]:
mdf.shape

(13852, 144)

In [132]:
# simplify df
sdf = df[ ['order_id','items_id']].copy()
sdf = sdf.drop_duplicates()
# remove orders with only one item
sdf = sdf.groupby('order_id').filter(lambda x: len(x) > 1)
sdf.reset_index(drop=True,inplace=True)
sdf.head(10)

,order_id,items_id
0,841222,2255
1,841222,3038
2,841222,2252
3,841222,3036
4,841237,2716
5,841237,2253
6,841237,2218
7,841345,2255
8,841345,2218
9,841393,3285


In [133]:
# count index through items for each order
sdf['count'] = sdf.groupby('order_id').cumcount()

In [134]:
sdf.head(10)

,order_id,items_id,count
0,841222,2255,0
1,841222,3038,1
2,841222,2252,2
3,841222,3036,3
4,841237,2716,0
5,841237,2253,1
6,841237,2218,2
7,841345,2255,0
8,841345,2218,1
9,841393,3285,0


In [135]:
pdf = pd.pivot_table(sdf,index="order_id",columns=['count'],values=['items_id'])

In [136]:
pdf.head(10)

items_id                                                              \
count          0       1       2       3   4   5   6   7   8   9   10  11  12   
order_id                                                                        
841222     2255.0  3038.0  2252.0  3036.0 NaN NaN NaN NaN NaN NaN NaN NaN NaN   
841237     2716.0  2253.0  2218.0     NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN   
841345     2255.0  2218.0     NaN     NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN   
841393     3285.0  2249.0  2252.0  3273.0 NaN NaN NaN NaN NaN NaN NaN NaN NaN   
841496     2253.0  3285.0  3287.0     NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN   
841542     3371.0  3273.0  2304.0  3274.0 NaN NaN NaN NaN NaN NaN NaN NaN NaN   
841656     3274.0  3371.0     NaN     NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN   
841677     3285.0  2252.0  3389.0  3411.0 NaN NaN NaN NaN NaN NaN NaN NaN NaN   
841714     3038.0  3370.0     NaN     NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN   
841736     3039.0  2255.0     NaN     NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN   

              
count     13  
order_id      
841222   NaN  
841237   NaN  
841345   NaN  
841393   NaN  
841496   NaN  
841542   NaN  
841656   NaN  
841677   NaN  
841714   NaN  
841736   NaN

In [137]:
pdf.shape

(11645, 14)

In [143]:
# make the numbers into strings and make the nans 'nan'
records = []  
for i in range(0, len(pdf)):
    jlist = []
    for j in range(0, pdf.shape[1]):        
        if not np.isnan(pdf.values[i,j]):
            xi = int(pdf.values[i,j])
            xi = str(xi)
        else:
            xi = str(pdf.values[i,j])
        jlist.append(xi)
    records.append(jlist)

In [144]:
print(records)

[['2255', '3038', '2252', '3036', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'], ['2716', '2253', '2218', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'], ['2255', '2218', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'], ['3285', '2249', '2252', '3273', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'], ['2253', '3285', '3287', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'], ['3371', '3273', '2304', '3274', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'], ['3274', '3371', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'], ['3285', '2252', '3389', '3411', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'], ['3038', '3370', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'], ['3039', '2255', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'

In [171]:
association_rules = apriori(records, min_support=0.001, min_confidence=0.1, min_lift=2, max_length=3)  
association_results = list(association_rules)  
print(len(association_results))

413


In [178]:
dfx = df_prods.copy()
dfx['items_id'] = dfx['items_id'].apply(str)
dfx.set_index('items_id',inplace=True)
for item in association_results:

    # first index of the inner list
    # Contains base item and add item
    #print(item)
    pair = item[0] 
    items = [x for x in pair]
    if items[0] == 'nan' or items[1] == 'nan':
        continue
    print("Rule: " + items[0] +" "+ dfx.loc[ items[0],'name'] + " -> " + items[1] + " "+ dfx.loc[ items[1],'name'])
#    print("Rule: " + items[0] + " -> " + items[1])

    #second index of the inner list
    print("Support: " + str(item[1]))

    #third index of the list located at 0th
    #of the third index of the inner list

    print("Confidence: " + str(item[2][0][2]))
    print("Lift: " + str(item[2][0][3]))
    print("=====================================")


Rule: 3236 Toy Story Land Cupcake -> 2207 No Sugar Added Strawberry Parfait
Support: 0.0016316015457277801
Confidence: 0.2209302325581395
Lift: 4.660747387933939
Rule: 2245 Olaf Cupcake -> 2208 Chocolate Mousse
Support: 0.0011163589523400601
Confidence: 0.14772727272727273
Lift: 9.054126794258373
Rule: 2208 Chocolate Mousse -> 3236 Toy Story Land Cupcake
Support: 0.0020609703735508804
Confidence: 0.27272727272727276
Lift: 5.753458498023716
Rule: 2221 Steak Fries -> 3274 Mac & Cheese
Support: 0.0015457277801631602
Confidence: 0.16071428571428573
Lift: 4.282649558679307
Rule: 3285 Southwest BBQ Burger -> 2221 Steak Fries
Support: 0.0016316015457277801
Confidence: 0.16964285714285715
Lift: 3.165851076007326
Rule: 2221 Steak Fries -> 3288 BBQ Rib Platter
Support: 0.0015457277801631602
Confidence: 0.16071428571428573
Lift: 3.4657738095238098
Rule: 2221 Steak Fries -> 3289 Southwestern Steak Fries
Support: 0.00120223271790468
Confidence: 0.125
Lift: 7.954234972677595
Rule: 2221 Steak Fries -

Support: 0.0010304851867754402
Confidence: 0.19672131147540986
Lift: 3.8180327868852464
Rule: 2255 Bud Light -> 2256 Yuengling
Support: 0.0018033490768570202
Confidence: 0.1337579617834395
Lift: 2.5960191082802546
Rule: 2255 Bud Light -> 3038 Shrimp, Fish and Chicken Platter
Support: 0.0012881064834693002
Confidence: 0.13761467889908258
Lift: 2.6708715596330275
Rule: 2255 Bud Light -> 2256 Yuengling
Support: 0.00120223271790468
Confidence: 0.13592233009708737
Lift: 2.638025889967637
Rule: 2255 Bud Light -> 2293 Woodbridge by Robert Mondavi Chardonnay
Support: 0.0012881064834693002
Confidence: 0.21126760563380284
Lift: 4.346663016971085
Rule: 2255 Bud Light -> 3276 Mac & Cheese
Support: 0.0012881064834693002
Confidence: 0.24590163934426232
Lift: 2.2044069208344377
Rule: 2255 Bud Light -> 3039 Happy Daze
Support: 0.00145985401459854
Confidence: 0.12686567164179105
Lift: 2.610160329096567
Rule: 2255 Bud Light -> 3265 Vegan Burger
Support: 0.0010304851867754402
Confidence: 0.10909090909090

In [162]:
dfx.head(5)

,name,display_desc,category,location,img
items_id,,,,,
1813,French Fries,NaN,Side,ABC Commissary,fries_french_fries.png
1818,Dannon® Greek Yogurt,Light and Fit Strawberry,Dessert,ABC Commissary,Dannon_Greek_LnF_Strawberry.png
1819,Vegetarian Chili,NaN,Side,ABC Commissary,chile_vegetarian.jpg
1820,New England Clam Chowder,NaN,Side,ABC Commissary,soup_NE_clam_chowder.png
1822,Grilled Salmon,served with Vegetable Rice and Green Beans and...,Entree,ABC Commissary,salmon_grilled_salmon_green_beans.jpg


In [184]:
print( sdf.loc[  sdf['items_id'] == 3288 ] )

       order_id  items_id  count
342      843374      3288      1
374      843784      3288      1
387      843820      3288      3
393      843836      3288      3
398      843840      3288      2
400      844085      3288      1
409      844128      3288      2
414      844329      3288      0
800      845979      3288      0
814      846081      3288      6
818      846090      3288      1
825      846094      3288      1
839      846196      3288      2
841      846209      3288      1
854      846329      3288      0
1207     847961      3288      1
1211     847969      3288      2
1217     847998      3288      4
1219     848097      3288      1
1225     848141      3288      2
1235     848218      3288      1
1236     848259      3288      0
1238     848263      3288      0
1251     848283      3288      1
1263     848304      3288      1
1267     848311      3288      2
1280     848328      3288      0
1294     848363      3288      1
1299     848387      3288      0
1303     8